<a href="https://colab.research.google.com/github/joynaomi81/Sentiment-Analysis-Project/blob/main/Movie_Reviews_Sentiment_Analysis_in_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [7]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [8]:
data.isnull().any()

review       False
sentiment    False
dtype: bool

In [9]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [12]:
data.shape

(50000, 2)

Test normalization

Tokenization

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt



In [14]:
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [15]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import wordcloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [16]:
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn. svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn. metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#Tokenization of text
tokenizers=ToktokTokenizer()
#Setting English stopwords
stopsword=nltk.corpus.stopwords.words('english')

In [19]:
#Removing the noisy text
def noiseremoval_text(text):
  soup = BeautifulSoup(text, 'html.parser')
  text = soup.get_text()
  text = re.sub('\[[^]]*\]','',text)
  return text

In [20]:
#Apply function on review column
data['review']=data['review'].apply(noiseremoval_text)

<ipython-input-19-96ff9fdc6d6d>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Stemming

In [22]:
#Stemming the text
def stemmer(text):
  ps=nltk.PorterStemmer()
  text= ''.join([ps.stem(word)for word in text.split()])
  return text

In [23]:
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

# Apply the stemming function to the 'review' column
data['review'] = data['review'].apply(stemmer)

print(data)

                                                  review sentiment
0      oneoftheotherreviewhamentionthatafterwatchjust...  positive
1      awonderlittlproduction.thefilmtechniquisveriun...  positive
2      ithoughtthiwaawonderwaytospendtimeonatoohotsum...  positive
3      basicthere'afamiliwherealittlboy(jake)thinkthe...  negative
4      pettermattei'"loveinthetimeofmoney"isavisualst...  positive
...                                                  ...       ...
49995  ithoughtthimovididadownrightgoodjob.itwasn'tas...  positive
49996  badplot,baddialogue,badacting,idiotdirecting,t...  negative
49997  iamacatholtaughtinparochielementarischoolbynun...  negative
49998  i'mgotohavetodisagrewiththeprevioucommentandsi...  negative
49999  nooneexpectthestartrekmovitobehighart,butthefa...  negative

[50000 rows x 2 columns]


In [24]:
data.head()

,review,sentiment
0,oneoftheotherreviewhamentionthatafterwatchjust...,positive
1,awonderlittlproduction.thefilmtechniquisveriun...,positive
2,ithoughtthiwaawonderwaytospendtimeonatoohotsum...,positive
3,basicthere'afamiliwherealittlboy(jake)thinkthe...,negative
4,"pettermattei'""loveinthetimeofmoney""isavisualst...",positive


Removing stop words

In [25]:
def removing_stopwords(text, is_lower_case=False):
    tokenizers=ToktokTokenizer()
    tokens = tokenizers.tokenize(text)
    tokens = [i.strip() for i in tokens]
    if is_lower_case:
        filtokens = [i for i in tokens if i not in stop_words]
    else:
         filtokens = [i for i in tokens if i.lower() not in stop_words]
    filtered_text = ' '.join(filtokens)
    return filtered_text
    data['text'] = data['text'].apply(removing_stopwords)



In [26]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

stop_words = stopwords.words("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
#Apply function on column
data['review']=data['review'].apply(removing_stopwords)

In [28]:
data.head()

,review,sentiment
0,oneoftheotherreviewhamentionthatafterwatchjust...,positive
1,awonderlittlproduction.thefilmtechniquisveriun...,positive
2,ithoughtthiwaawonderwaytospendtimeonatoohotsum...,positive
3,basicthere ' afamiliwherealittlboy( jake ) thi...,negative
4,"pettermattei ' "" loveinthetimeofmoney "" isavis...",positive


# Train test split

In [29]:
#test dataset
train_reviews_data=data.review[:30000]

In [30]:
test_reviews_data=data.review[30000:]

In [31]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=1,max_df=2,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train=cv.fit_transform(train_reviews_data)
cv_test=cv.transform(test_reviews_data)

print('BOW_cv_train:',cv_train.shape)
print('BOW_cv_test:',cv_test.shape)


BOW_cv_train: (30000, 2778550)
BOW_cv_test: (20000, 2778550)


## **TF_IDF**

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(min_df=1,max_df=2,binary=False,ngram_range=(1,3))
tf_train=tf.fit_transform(train_reviews_data)
tf_test=tf.transform(test_reviews_data)
print('BOW_Tfid_train:',tf_train.shape)
print('BOW_Tfid_test:',tf_test.shape)

BOW_Tfid_train: (30000, 2778550)
BOW_Tfid_test: (20000, 2778550)


## **Lable encoding**

In [33]:
#labeling the sentiment data
label=LabelBinarizer()
#transformed sentiment
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [34]:
train_data=data.sentiment[:30000]

In [35]:
test_data=data.sentiment[30000:]

In [36]:
sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [37]:
train_data=data.sentiment[:30000]

In [38]:
test_data=data.sentiment[30000:]

# Logistic Regression

In [39]:
#training the model
logistic=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#fitting the model for bag of words
lr_bow = logistic.fit(cv_train, train_data)
print(lr_bow)
#fitting the model for tfidf features
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [40]:
#Predicting the model for bag of words
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [41]:
bow_predict=logistic.predict(cv_test)
print(bow_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [42]:
bow_score=accuracy_score(test_data,bow_predict)
print('lr_bow_score:',bow_score)

lr_bow_score: 0.6404


In [43]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [44]:
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print('lr_bow_score:',lr_bow_predict)

lr_bow_score: ['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [45]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [46]:
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print('lr_tfidf_score:',lr_tfidf_score)

lr_tfidf_score: 0.6405


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
cv = TfidfVectorizer(max_features=5000)
x = cv.fit_transform(data['review']).toarray()
y=  data['sentiment']

In [48]:
x.shape

(50000, 5000)

In [49]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [50]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

((40000, 5000), (10000, 5000), (40000,), (10000,))

## Naive Bayes Classifier

In [51]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [52]:
mnb = MultinomialNB()
mnb.fit(x_train, y_train)

MultinomialNB()

In [53]:
prediction = mnb.predict(x_test)

In [54]:
accuracy = accuracy_score(y_test, prediction)
print(f'Accuracy: {accuracy}')

print('\nClassification Report:')
print(classification_report(y_test, prediction))

print('\nConfusion Matrix:')
print(confusion_matrix(y_test, prediction))

Accuracy: 0.6871

Classification Report:
              precision    recall  f1-score   support

    negative       0.67      0.74      0.70      4961
    positive       0.71      0.63      0.67      5039

    accuracy                           0.69     10000
   macro avg       0.69      0.69      0.69     10000
weighted avg       0.69      0.69      0.69     10000


Confusion Matrix:
[[3676 1285]
 [1844 3195]]


In [55]:
comparison_df = pd.DataFrame(np.c_[y_test, prediction], columns=['Actual', 'Predicted'])

print(comparison_df)

        Actual Predicted
0     positive  positive
1     positive  positive
2     negative  negative
3     positive  positive
4     negative  negative
...        ...       ...
9995  negative  negative
9996  positive  positive
9997  positive  positive
9998  negative  positive
9999  positive  negative

[10000 rows x 2 columns]


In [72]:
import os

# Print current working directory
print("/content/cv-vectorzier.plk:", os.getcwd())

# List files in the current working directory
print("Files in the current working directory:", os.listdir())


/content/cv-vectorzier.plk: /content
Files in the current working directory: ['.config', 'Movie_Reviews_Sentiment_Analysis.pkl', 'Count-vectorizer.pkl', 'drive', 'cv-vectorizer.pkl', 'Movies_Review_Sentiment_Analysis.pkl', 'sample_data']


In [71]:
import pickle  # Import the pickle module

# Save the CountVectorizer and the model
pickle.dump(cv, open('Count-vectorizer.pkl', 'wb'))
pickle.dump(mnb, open('Movie_Reviews_Sentiment_Analysis.pkl', 'wb'))


In [62]:
def test_model(sentence):
  sentence = save_cv.transform([sentence])
  predicted_sentiment = model.predict(sentence)[0]
  return predicted_sentiment

In [73]:
import pickle

# Load the CountVectorizer and model from pickle files
save_cv = pickle.load(open('Count-vectorizer.pkl', 'rb'))
model = pickle.load(open('Movie_Reviews_Sentiment_Analysis.pkl', 'rb'))


In [65]:
sentence = 'This is the best movie  of my life ever'
predicted_sentiment = test_model(sentence)
print(predicted_sentiment)

positive


In [66]:
sentence = 'This is the worse movie  of my life ever'
predicted_sentiment = test_model(sentence)
print(predicted_sentiment)

negative


In [67]:
sentence = 'I hate the movie '
predicted_sentiment = test_model(sentence)
print(predicted_sentiment)

negative
